In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    
    #check if image is grayscale or color
    shape = img.shape
    if(len(shape) == 3):               #its a color image
        mask_color = (255,)*shape[-1]   #shape[-1] = no. at last index
    else:                              #otherwise its a gray image
        mask_color = 255
      
    # Fill the polygon with white
    cv2.fillPoly(mask, vertices, mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [3]:
def applySobel(img, orient = 'x', kernel=3, th=100, blackWhite=(0, 255)):
    #cvt to gray
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #extract vertical or horizontal line
    if orient == 'x':
        absSobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0))
    else:
        absSobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1))
        
    #rescale to 8 bit
    scaled = np.uint8(255*absSobel/np.max(absSobel))
    
    #create a blank image and apply thresh
    op = np.zeros_like(scaled)
    op[scaled > th] = blackWhite[1] #white
    op[scaled < th] = blackWhite[0] #black
    
    return op

In [4]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def filter_color(img):
    yellow_min = np.array([50, 80, 80], np.uint8)
    yellow_max = np.array([60, 255, 255], np.uint8)
    yellow_mask = cv2.inRange(img, yellow_min, yellow_max)

    white_min = np.array([200, 0, 200], np.uint8)
    white_max = np.array([255, 0, 255], np.uint8)
    white_mask = cv2.inRange(img, white_min, white_max)
    
    yelWhtMerge = cv2.bitwise_or(yellow_mask, white_mask)
    img = cv2.bitwise_and(img, img, yelWhtMerge)
    return img

In [8]:
def pipeline(frame):
    #get height and width of frame
    height, width, channel = frame.shape

    #keeping the portion of image which contains lane
    topLeftPt = (width*0.30, height*0.60)
    topRightPt = (width*0.70, height*0.60)

    #this portion contain road
    region_of_interest_points = [
    (0, height*0.90),
    #(0, height*0.60),
    topLeftPt,
    topRightPt,
    #(width, height*0.60),
    (width, height*0.90),
    ]
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = cv2.Canny(img, 80, 185)
    img = applySobel(img, 'x', 3)
    img = region_of_interest(img, np.array([region_of_interest_points], np.int32))
    return img

# process and save video

In [9]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [10]:
laneDay = '../dataset/day-lane.mp4'
laneDayOut = './dataset/laneDayOut.mp4'
laneVideo = VideoFileClip(laneDay)
output = laneVideo.fl_image(pipeline) #NOTE: this function expects color images!!
%time output.write_videofile(laneDayOut, audio=False)

[MoviePy] >>>> Building video ../dataset/laneDayOut.mp4
[MoviePy] Writing video ../dataset/laneDayOut.mp4


100%|█████████▉| 13290/13291 [14:49<00:00, 12.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../dataset/laneDayOut.mp4 

CPU times: user 7min 1s, sys: 1min 56s, total: 8min 58s
Wall time: 14min 53s


In [13]:
HTML("""<video width="640" height="480" controls> <source src="{0}"></video>""".format(laneDayOut))

In [14]:
cap = cv2.VideoCapture('../dataset/day-lane.mp4')
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('./dataset/histSobelx.avi', fourcc, 20, (480,640), isColor='True')

#get height and width of frame
ret, frame = cap.read()
height, width, channel = frame.shape

#keeping the portion of image which contains lane
topLeftPt = (width*0.30, height*0.60)
topRightPt = (width*0.70, height*0.60)

#this portion contain road
region_of_interest_points = [
(0, height*0.90),
#(0, height*0.60),
topLeftPt,
topRightPt,
#(width, height*0.60),
(width, height*0.90),
]

i = 1  #image count
while(cap.isOpened()):
    # Take each frame
    ret, frame = cap.read()
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = cv2.Canny(img, 80, 185)
    img = applySobel(img, 'x', 3)
    img = region_of_interest(img, np.array([region_of_interest_points], np.int32))
    cv2.imshow('img', img)
    #out.write(img)
    k = cv2.waitKey(25) & 0xFF
    if k == ord('q'):
        break
    elif k == ord('s'):
        cv2.imwrite('dataset/ld'+str(i)+'.jpg', frame)
        i += 1

cap.release()
#out.release()
cv2.destroyAllWindows()

# Custom color

In [28]:
img = np.zeros((15,15,3), dtype = np.uint8)

img[:,:,1] = 255   #saturation
img[:,:,2] = 255   #value
j = 1
#changing hue
for i in range(0, 256, 100):
    img[:,:,0] = i
    #cvt to rgb
    color = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.imshow('y', color)
    j += 1
    